In [27]:
from azureml.core import Workspace
ws = Workspace.from_config()

In [28]:
from azureml.core import Experiment
experiment = Experiment(workspace=ws, name="diabetes_classification")

In [29]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Dataset

dataset = Dataset.get_by_name(ws, name='diabetes_classification')
df = dataset.to_pandas_dataframe()

In [30]:
df.isna().sum()

p_id                     0
no_times_pregnant        0
glucose_concentration    0
blood_pressure           0
skin_fold_thickness      0
serum_insulin            0
bmi                      0
diabetes pedigree        0
age                      0
diabetes                 0
dtype: int64

In [31]:
X = df.iloc[: , :-1]
y = df.iloc[: , -1]

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=66)

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

lr=LogisticRegression()
lr.fit(X_train, y_train)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [34]:
y_pred=lr.predict(X_train)

In [35]:
from sklearn import metrics
print("Accuracy Score:-", metrics.accuracy_score(y_train, y_pred))
print("F1 Score:-", metrics.f1_score(y_train, y_pred))
print("Average Precision Score:-", metrics.average_precision_score(y_train, y_pred))
print("Log Loss:-", metrics.log_loss(y_train, y_pred))
print("Precision Score:-", metrics.precision_score(y_train, y_pred))
print("Recall Score:-", metrics.recall_score(y_train, y_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_train, y_pred))

Accuracy Score:- 0.7087576374745418
F1 Score:- 0.4837545126353791
Average Precision Score:- 0.44803180160693246
Log Loss:- 10.059224861836508
Precision Score:- 0.6090909090909091
Recall Score:- 0.40119760479041916
ROC-AUC Score:- 0.6342407777038515


In [ ]:
import joblibmodel_file_name = 'diabetes_logistic_regression.pkl'

# save model in the outputs folder so it automatically get uploaded
with open(model_file_name, "wb") as file:
    joblib.dump(value=reg, filename=os.path.join('./outputs/',
                                                    model_file_name))

In [1]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from azureml.core.run import Run
import os
import numpy as np
from sklearn import __version__ as sklearnver
from packaging.version import Version
if Version(sklearnver) < Version("0.23.0"):
    from sklearn.externals import joblib
else:
    import joblib

os.makedirs('./outputs', exist_ok=True)

X, y = load_diabetes(return_X_y=True)

run = Run.get_context()

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=0)
data = {"train": {"X": X_train, "y": y_train},
        "test": {"X": X_test, "y": y_test}}

# list of numbers from 0.0 to 1.0 with a 0.05 interval
alphas = np.arange(0.0, 1.0, 0.05)

for alpha in alphas:
    # Use Ridge algorithm to create a regression model
    reg = Ridge(alpha=alpha)
    reg.fit(data["train"]["X"], data["train"]["y"])

    preds = reg.predict(data["test"]["X"])
    mse = mean_squared_error(preds, data["test"]["y"])
    run.log('alpha', alpha)
    run.log('mse', mse)

    model_file_name = 'ridge_{0:.2f}.pkl'.format(alpha)
    # save model in the outputs folder so it automatically get uploaded
    with open(model_file_name, "wb") as file:
        joblib.dump(value=reg, filename=os.path.join('./outputs/',
                                                     model_file_name))

    print('alpha is {0:.2f}, and mse is {1:0.2f}'.format(alpha, mse))


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Attempted to log scalar metric alpha:
0.0
Attempted to log scalar metric mse:
3424.3166882137343
alpha is 0.00, and mse is 3424.32
Attempted to log scalar metric alpha:
0.05
Attempted to log scalar metric mse:
3408.9153122589296
alpha is 0.05, and mse is 3408.92
Attempted to log scalar metric alpha:
0.1
Attempted to log scalar metric mse:
3372.649627810032
alpha is 0.10, and mse is 3372.65
Attempted to log scalar metric alpha:
0.15000000000000002
Attempted to log scalar metric mse:
3345.1496434741894
alpha is 0.15, and mse is 3345.15
Attempted to log scalar metric alpha:
0.2
Attempted to log scalar metric mse:
3325.2946794678764
alpha is 0.20, and mse is 3325.29
Attempted to log scalar metric alpha:
0.25
Attempted to log scalar metric mse:
3311.5562509289744
alpha is 0.25, and mse is 3311.56
Attempted to log scalar metric alpha:
0.30000000000000004
Attempted to log scalar metric mse:
3302.6736334017255
alpha is 0.30, and mse is 3302.67
Attempted to log scalar metric alpha:
0.3500000000

In [2]:
from azureml.core import Workspace
ws = Workspace.from_config()

In [3]:
from azureml.core.model import Model

# Register model
model = Model.register(ws, model_name="sklearn-diabetes", model_path="./outputs/ridge_0.30.pkl")

Registering model sklearn-diabetes


In [4]:
%%writefile score.py
import json
import os
import joblib

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType

def init():
    print("This is init")
    global model

    # The AZUREML_MODEL_DIR environment variable indicates
    # a directory containing the model file you registered.
    model_filename = 'ridge_0.30.pkl'
    model_path = os.path.join(os.environ['AZUREML_MODEL_DIR'], model_filename)

    model = joblib.load(model_path)



@input_schema('data', NumpyParameterType(np.array([[0.1, 1.2, 2.3, 3.4, 4.5, 5.6, 6.7, 7.8, 8.9, 9.0]])))
@output_schema(NumpyParameterType(np.array([4429.929236457418])))
def run(data):
    # Use the model object loaded by init().
    result = model.predict(data)

    # You can return any JSON-serializable object.
    return result.tolist()

Overwriting score.py


In [142]:
def run(data):
    model = joblib.load("outputs/ridge_0.30.pkl")
    # test = json.loads(data)
    result = model.predict(data)
    return result.tolist()

In [146]:
run([[ 0.01991321,  0.05068012,  0.10480869,  0.07007254, -0.03596778,-0.0266789 , -0.02499266, -0.00259226,  0.00371174,  0.04034337]])

[221.45692200623236]

In [130]:
data["test"]["X"][0]

array([ 0.01991321,  0.05068012,  0.10480869,  0.07007254, -0.03596778,
       -0.0266789 , -0.02499266, -0.00259226,  0.00371174,  0.04034337])

In [124]:
import joblib

[data["test"]["X"][0:5]]

[array([[ 0.01991321,  0.05068012,  0.10480869,  0.07007254, -0.03596778,
         -0.0266789 , -0.02499266, -0.00259226,  0.00371174,  0.04034337],
        [-0.01277963, -0.04464164,  0.06061839,  0.05285819,  0.04796534,
          0.02937467, -0.01762938,  0.03430886,  0.0702113 ,  0.00720652],
        [ 0.03807591,  0.05068012,  0.00888341,  0.04252958, -0.04284755,
         -0.02104223, -0.03971921, -0.00259226, -0.01811827,  0.00720652],
        [-0.01277963, -0.04464164, -0.02345095, -0.04009932, -0.01670444,
          0.00463594, -0.01762938, -0.00259226, -0.03845911, -0.03835666],
        [-0.02367725, -0.04464164,  0.04552903,  0.09072977, -0.01808039,
         -0.03544706,  0.07072993, -0.03949338, -0.03452372, -0.00936191]])]

In [5]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name="AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference")
inference_config = InferenceConfig(
    environment=env,
    entry_script="score.py",
)

In [6]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=0.5, memory_gb=1, auth_enabled=True
)

In [7]:
service = Model.deploy(
    ws,
    "sklearn-diabetes",
    [model],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-12-07 17:41:14+00:00 Creating Container Registry if not exists.
2021-12-07 17:41:14+00:00 Registering the environment.
2021-12-07 17:41:15+00:00 Use the existing image.
2021-12-07 17:41:15+00:00 Generating deployment configuration.
2021-12-07 17:41:16+00:00 Submitting deployment to compute.
2021-12-07 17:41:19+00:00 Checking the status of deployment sklearn-diabetes..
2021-12-07 17:51:43+00:00 Checking the status of inference endpoint sklearn-diabetes.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 2f66ddd9-ad46-40d1-a9d9-8cb20fa29d78
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: sklearn-diabetes. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image viennaglobal.azurecr.io/azureml/azureml_69581a78861b814a11058f72f478b9e0 locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more inform

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 2f66ddd9-ad46-40d1-a9d9-8cb20fa29d78
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: sklearn-diabetes. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image viennaglobal.azurecr.io/azureml/azureml_69581a78861b814a11058f72f478b9e0 locally. Please refer to https://ak

In [ ]:
model = joblib.load(model_path)

In [148]:
service

AciWebservice(workspace=Workspace.create(name='azure-machine-learning', subscription_id='29b64be4-867b-40ee-a259-b58b97bfc26f', resource_group='99-azure-machine-learning'), name=sklearn-diabetes, image_id=None, compute_type=None, state=ACI, scoring_uri=Healthy, tags=http://101c3820-712d-4913-b665-c4948751a509.westeurope.azurecontainer.io/score, properties=None, created_by={'hasInferenceSchema': 'False', 'hasHttps': 'False'})

In [153]:
input_payload = json.dumps({
    'data': [[0.01991321,  0.05068012,  0.10480869,  0.07007254, -0.03596778,-0.0266789 , -0.02499266, -0.00259226,  0.00371174,  0.04034337]]
})

In [154]:
service.run(input_payload)

Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '329', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Tue, 07 Dec 2021 17:27:02 GMT', 'Server': 'nginx/1.14.0 (Ubuntu)', 'X-Ms-Request-Id': 'bb5bae0c-2af0-4736-af20-bcbaad0a873e', 'X-Ms-Run-Function-Failed': 'True'}
Content: b'Expected 2D array, got scalar array instead:\narray={"data": [[0.01991321, 0.05068012, 0.10480869, 0.07007254, -0.03596778, -0.0266789, -0.02499266, -0.00259226, 0.00371174, 0.04034337]]}.\nReshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.'



WebserviceException: WebserviceException:
	Message: Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '329', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Tue, 07 Dec 2021 17:27:02 GMT', 'Server': 'nginx/1.14.0 (Ubuntu)', 'X-Ms-Request-Id': 'bb5bae0c-2af0-4736-af20-bcbaad0a873e', 'X-Ms-Run-Function-Failed': 'True'}
Content: b'Expected 2D array, got scalar array instead:\narray={"data": [[0.01991321, 0.05068012, 0.10480869, 0.07007254, -0.03596778, -0.0266789, -0.02499266, -0.00259226, 0.00371174, 0.04034337]]}.\nReshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.'
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.